# एक प्रतिगमन मॉडल बनाएं: डेटा तैयार करें और विज़ुअलाइज़ करें

## **कद्दू के लिए रैखिक प्रतिगमन - पाठ 2**
#### परिचय

अब जब आपके पास मशीन लर्निंग मॉडल बनाने के लिए Tidymodels और Tidyverse के साथ काम करने के उपकरण तैयार हैं, तो आप अपने डेटा से सवाल पूछने के लिए तैयार हैं। जब आप डेटा के साथ काम करते हैं और एमएल समाधान लागू करते हैं, तो यह समझना बहुत महत्वपूर्ण है कि सही सवाल कैसे पूछा जाए ताकि आपके डेटा सेट की संभावनाओं को सही तरीके से खोला जा सके।

इस पाठ में, आप सीखेंगे:

-   मॉडल-निर्माण के लिए अपने डेटा को कैसे तैयार करें।

-   डेटा विज़ुअलाइज़ेशन के लिए `ggplot2` का उपयोग कैसे करें।

आपके द्वारा पूछे जाने वाले सवाल यह तय करेंगे कि आप किस प्रकार के एमएल एल्गोरिदम का उपयोग करेंगे। और आपको जो उत्तर मिलेगा उसकी गुणवत्ता काफी हद तक आपके डेटा की प्रकृति पर निर्भर करेगी।

आइए इसे एक व्यावहारिक अभ्यास के माध्यम से समझते हैं।

<p >
   <img src="../../images/unruly_data.jpg"
   width="700"/>
   <figcaption>@allison_horst द्वारा कलाकृति</figcaption>


<!--![\@allison_horst द्वारा कलाकृति](../../../../../../translated_images/unruly_data.0eedc7ced92d2d919cf5ea197bfe0fe9a30780c4bf7cdcf14ff4e9dc5a4c7267.hi.jpg)<br>\@allison_horst द्वारा कलाकृति-->


## 1. कद्दू डेटा आयात करना और Tidyverse को बुलाना

इस पाठ को समझने और डेटा को व्यवस्थित करने के लिए हमें निम्नलिखित पैकेजों की आवश्यकता होगी:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) एक [R पैकेजों का संग्रह](https://www.tidyverse.org/packages) है, जिसे डेटा साइंस को तेज़, आसान और मज़ेदार बनाने के लिए डिज़ाइन किया गया है!

आप इन्हें इस तरह से इंस्टॉल कर सकते हैं:

`install.packages(c("tidyverse"))`

नीचे दिया गया स्क्रिप्ट यह जांचता है कि क्या आपके पास इस मॉड्यूल को पूरा करने के लिए आवश्यक पैकेज हैं, और यदि कुछ गायब हैं तो उन्हें आपके लिए इंस्टॉल कर देता है।


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse)

अब, कुछ पैकेज शुरू करें और इस पाठ के लिए प्रदान किए गए [डेटा](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/data/US-pumpkins.csv) को लोड करें!


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n =50)

`glimpse()` का उपयोग करने पर तुरंत पता चलता है कि डेटा में खाली स्थान हैं और स्ट्रिंग्स (`chr`) और संख्यात्मक डेटा (`dbl`) का मिश्रण है। `Date` का प्रकार कैरेक्टर है और एक अजीब कॉलम `Package` भी है, जिसमें डेटा `sacks`, `bins` और अन्य मानों का मिश्रण है। वास्तव में, डेटा थोड़ा गड़बड़ है 😤।

सच कहें तो, ऐसा बहुत कम होता है कि आपको एक ऐसा डेटासेट मिले जो पूरी तरह से तैयार हो और जिसे सीधे ML मॉडल बनाने के लिए उपयोग किया जा सके। लेकिन चिंता मत करें, इस पाठ में आप सीखेंगे कि कैसे एक कच्चे डेटासेट को मानक R लाइब्रेरीज़ 🧑‍🔧 का उपयोग करके तैयार किया जाए। आप डेटा को विज़ुअलाइज़ करने की विभिन्न तकनीकों को भी सीखेंगे।📈📊
<br>

> एक पुनश्चर्या: पाइप ऑपरेटर (`%>%`) लॉजिकल क्रम में ऑपरेशन्स करता है, जिसमें एक ऑब्जेक्ट को आगे किसी फंक्शन या कॉल एक्सप्रेशन में पास किया जाता है। आप पाइप ऑपरेटर को अपने कोड में "और फिर" कहने जैसा समझ सकते हैं।


## 2. डेटा में कमी की जांच करें

डेटा वैज्ञानिकों को जिन सबसे आम समस्याओं का सामना करना पड़ता है, उनमें से एक है अधूरी या गायब डेटा। R गायब या अज्ञात मानों को एक विशेष संकेतक मान `NA` (Not Available) के रूप में दर्शाता है।

तो हम कैसे जानेंगे कि डेटा फ्रेम में गायब मान हैं?  
<br>
-   एक सीधा तरीका यह होगा कि बेस R फ़ंक्शन `anyNA` का उपयोग करें, जो तार्किक वस्तुएं `TRUE` या `FALSE` लौटाता है।


In [ ]:
pumpkins %>% 
  anyNA()

यह सही है, ऐसा लगता है कि कुछ डेटा गायब है! यह शुरू करने के लिए एक अच्छा स्थान है।

-   एक और तरीका यह हो सकता है कि `is.na()` फ़ंक्शन का उपयोग करें, जो यह दर्शाता है कि कौन से व्यक्तिगत कॉलम तत्व एक तार्किक `TRUE` के साथ गायब हैं।


In [ ]:
pumpkins %>% 
  is.na() %>% 
  head(n = 7)

ठीक है, इतना बड़ा डेटा फ्रेम होने पर, हर पंक्ति और स्तंभ को व्यक्तिगत रूप से जांचना अक्षम और व्यावहारिक रूप से असंभव होगा😴।

-   एक अधिक सहज तरीका यह होगा कि प्रत्येक स्तंभ के लिए गायब मानों का योग निकालें:


In [ ]:
pumpkins %>% 
  is.na() %>% 
  colSums()

बहुत बेहतर! कुछ डेटा गायब है, लेकिन शायद यह दिए गए कार्य के लिए मायने नहीं रखेगा। चलिए देखते हैं कि आगे का विश्लेषण क्या सामने लाता है।

> शानदार पैकेज और फंक्शन्स के साथ-साथ, R में बहुत अच्छी डॉक्यूमेंटेशन है। उदाहरण के लिए, `help(colSums)` या `?colSums` का उपयोग करके इस फंक्शन के बारे में अधिक जानकारी प्राप्त करें।


## 3. Dplyr: डेटा को व्यवस्थित करने का व्याकरण

<p >
   <img src="../../images/dplyr_wrangling.png"
   width="569"/>
   <figcaption>चित्रांकन: @allison_horst</figcaption>


<!--![चित्रांकन: \@allison_horst](../../../../../../translated_images/dplyr_wrangling.f5f99c64fd4580f1377fee3ea428b6f8fd073845ec0f8409d483cfe148f0984e.hi.png)<br/>चित्रांकन: \@allison_horst-->


[`dplyr`](https://dplyr.tidyverse.org/), Tidyverse का एक पैकेज है, जो डेटा प्रबंधन की व्याकरण प्रदान करता है और एक समान क्रियाओं का सेट देता है जो आपको सबसे सामान्य डेटा प्रबंधन चुनौतियों को हल करने में मदद करता है। इस भाग में, हम dplyr की कुछ क्रियाओं का अन्वेषण करेंगे!  
<br>


#### dplyr::select()

`select()` `dplyr` पैकेज में एक फ़ंक्शन है जो आपको कॉलम चुनने या हटाने में मदद करता है।

अपने डेटा फ्रेम को अधिक सुविधाजनक बनाने के लिए, `select()` का उपयोग करके उन कॉलम्स को रखें जो आपको चाहिए और बाकी को हटा दें।

उदाहरण के लिए, इस अभ्यास में हमारा विश्लेषण `Package`, `Low Price`, `High Price` और `Date` कॉलम्स पर आधारित होगा। आइए इन कॉलम्स को चुनें।


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(Package, `Low Price`, `High Price`, Date)


# Print data set
pumpkins %>% 
  slice_head(n = 5)

#### dplyr::mutate()

`mutate()` `dplyr` पैकेज का एक फ़ंक्शन है, जो आपको नए कॉलम बनाने या मौजूदा कॉलम को संशोधित करने में मदद करता है, जबकि बाकी कॉलम को वैसा ही रखता है।

`mutate` का सामान्य ढांचा इस प्रकार है:

`data %>%   mutate(new_column_name = what_it_contains)`

आइए `mutate` का उपयोग करके `Date` कॉलम पर निम्नलिखित कार्य करें:

1. तारीखों को (जो वर्तमान में character प्रकार की हैं) महीने के प्रारूप में बदलें (ये अमेरिकी तारीखें हैं, इसलिए प्रारूप `MM/DD/YYYY` है)।

2. तारीखों से महीने को निकालकर एक नए कॉलम में डालें।

R में, [lubridate](https://lubridate.tidyverse.org/) पैकेज Date-time डेटा के साथ काम करना आसान बनाता है। तो, आइए `dplyr::mutate()`, `lubridate::mdy()`, `lubridate::month()` का उपयोग करें और ऊपर दिए गए उद्देश्यों को पूरा करें। हम Date कॉलम को हटा सकते हैं क्योंकि हमें इसे आगे के कार्यों में उपयोग करने की आवश्यकता नहीं होगी।


In [ ]:
# Load lubridate
library(lubridate)

pumpkins <- pumpkins %>% 
  # Convert the Date column to a date object
  mutate(Date = mdy(Date)) %>% 
  # Extract month from Date
  mutate(Month = month(Date)) %>% 
  # Drop Date column
  select(-Date)

# View the first few rows
pumpkins %>% 
  slice_head(n = 7)

वाह! 🤩

अब, आइए एक नया कॉलम `Price` बनाते हैं, जो कद्दू की औसत कीमत को दर्शाता है। अब, `Low Price` और `High Price` कॉलम का औसत लेकर नए Price कॉलम को भरते हैं।  
<br>


In [ ]:
# Create a new column Price
pumpkins <- pumpkins %>% 
  mutate(Price = (`Low Price` + `High Price`)/2)

# View the first few rows of the data
pumpkins %>% 
  slice_head(n = 5)

येस!💪

"लेकिन रुको!", आप कहेंगे जब आप पूरे डेटा सेट को `View(pumpkins)` के साथ जल्दी से देखेंगे, "यहाँ कुछ अजीब है!"🤔

अगर आप `Package` कॉलम को देखें, तो कद्दू कई अलग-अलग विन्यासों में बेचे जाते हैं। कुछ `1 1/9 bushel` माप में बेचे जाते हैं, कुछ `1/2 bushel` माप में, कुछ प्रति कद्दू, कुछ प्रति पाउंड, और कुछ बड़े बॉक्सों में जिनकी चौड़ाई अलग-अलग होती है।

आइए इसे सत्यापित करें:


In [ ]:
# Verify the distinct observations in Package column
pumpkins %>% 
  distinct(Package)

अद्भुत!👏

कद्दू का वजन लगातार मापना काफी मुश्किल लगता है, तो चलिए उन्हें छांटते हैं। इसके लिए, `Package` कॉलम में केवल उन कद्दुओं को चुनें जिनमें *bushel* शब्द हो और इसे एक नए डेटा फ्रेम `new_pumpkins` में रखें।


#### dplyr::filter() और stringr::str_detect()

[`dplyr::filter()`](https://dplyr.tidyverse.org/reference/filter.html): डेटा का एक उपसमूह बनाता है जिसमें केवल वे **पंक्तियाँ** शामिल होती हैं जो आपकी शर्तों को पूरा करती हैं। इस मामले में, `Package` कॉलम में *bushel* स्ट्रिंग वाले कद्दू।

[stringr::str_detect()](https://stringr.tidyverse.org/reference/str_detect.html): किसी स्ट्रिंग में किसी पैटर्न की उपस्थिति या अनुपस्थिति का पता लगाता है।

[`stringr`](https://github.com/tidyverse/stringr) पैकेज सामान्य स्ट्रिंग ऑपरेशन्स के लिए सरल फ़ंक्शन प्रदान करता है।


In [ ]:
# Retain only pumpkins with "bushel"
new_pumpkins <- pumpkins %>% 
       filter(str_detect(Package, "bushel"))

# Get the dimensions of the new data
dim(new_pumpkins)

# View a few rows of the new data
new_pumpkins %>% 
  slice_head(n = 5)

आप देख सकते हैं कि हमने लगभग 415 पंक्तियों के डेटा को सीमित कर दिया है, जिसमें कद्दू बसों में शामिल हैं।🤩  
<br>


#### dplyr::case_when()

**लेकिन रुको! अभी एक और काम बाकी है**

क्या आपने देखा कि प्रति पंक्ति बसल की मात्रा अलग-अलग है? आपको मूल्य निर्धारण को सामान्य करना होगा ताकि आप प्रति बसल मूल्य दिखा सकें, न कि 1 1/9 या 1/2 बसल के लिए। इसे मानकीकृत करने के लिए अब कुछ गणना करनी होगी।

हम [`case_when()`](https://dplyr.tidyverse.org/reference/case_when.html) फ़ंक्शन का उपयोग करेंगे ताकि कुछ शर्तों के आधार पर Price कॉलम को *mutate* किया जा सके। `case_when` आपको कई `if_else()` स्टेटमेंट्स को वेक्टराइज़ करने की सुविधा देता है।


In [ ]:
# Convert the price if the Package contains fractional bushel values
new_pumpkins <- new_pumpkins %>% 
  mutate(Price = case_when(
    str_detect(Package, "1 1/9") ~ Price/(1 + 1/9),
    str_detect(Package, "1/2") ~ Price/(1/2),
    TRUE ~ Price))

# View the first few rows of the data
new_pumpkins %>% 
  slice_head(n = 30)

अब हम उनके बुशल माप के आधार पर प्रति इकाई मूल्य का विश्लेषण कर सकते हैं। हालांकि, कद्दू के बुशल का यह सारा अध्ययन यह दिखाने के लिए है कि `अपने डेटा की प्रकृति को समझना` कितना `महत्वपूर्ण` है!

> ✅ [The Spruce Eats](https://www.thespruceeats.com/how-much-is-a-bushel-1389308) के अनुसार, बुशल का वजन उत्पाद के प्रकार पर निर्भर करता है, क्योंकि यह एक आयतन माप है। "उदाहरण के लिए, टमाटर का एक बुशल 56 पाउंड वजन का होता है... पत्तियां और साग अधिक जगह लेते हैं लेकिन वजन कम होता है, इसलिए पालक का एक बुशल केवल 20 पाउंड का होता है।" यह सब काफी जटिल है! चलिए बुशल-से-पाउंड रूपांतरण करने की झंझट में न पड़ें और इसके बजाय बुशल के आधार पर मूल्य निर्धारण करें। कद्दू के बुशल का यह सारा अध्ययन, हालांकि, यह दिखाने के लिए है कि अपने डेटा की प्रकृति को समझना कितना महत्वपूर्ण है!
>
> ✅ क्या आपने देखा कि आधे-बुशल में बिकने वाले कद्दू बहुत महंगे हैं? क्या आप इसका कारण समझ सकते हैं? संकेत: छोटे कद्दू बड़े कद्दुओं की तुलना में बहुत महंगे होते हैं, शायद इसलिए कि एक बड़े खोखले पाई कद्दू द्वारा ली गई जगह के कारण प्रति बुशल उनमें बहुत अधिक छोटे कद्दू होते हैं।


अब अंत में, सिर्फ रोमांच के लिए 💁‍♀️, चलिए Month कॉलम को पहले स्थान पर ले चलते हैं, यानी `Package` कॉलम से पहले।

कॉलम की पोजीशन बदलने के लिए `dplyr::relocate()` का उपयोग किया जाता है।


In [ ]:
# Create a new data frame new_pumpkins
new_pumpkins <- new_pumpkins %>% 
  relocate(Month, .before = Package)

new_pumpkins %>% 
  slice_head(n = 7)

अच्छा काम!👌 अब आपके पास एक साफ-सुथरा और व्यवस्थित डेटा सेट है, जिस पर आप अपना नया रिग्रेशन मॉडल बना सकते हैं!  
<br>


## 4. ggplot2 के साथ डेटा विज़ुअलाइज़ेशन

<p >
   <img src="../../images/data-visualization.png"
   width="600"/>
   <figcaption>डसानी मदीपल्ली द्वारा इन्फोग्राफिक</figcaption>


<!--![डसानी मदीपल्ली द्वारा इन्फोग्राफिक](../../../../../../translated_images/data-visualization.54e56dded7c1a804d00d027543f2881cb32da73aeadda2d4a4f10f3497526114.hi.png){width="600"}-->

एक *समझदार* कहावत है जो इस प्रकार है:

> "एक साधारण ग्राफ ने डेटा विश्लेषक के दिमाग में किसी भी अन्य उपकरण की तुलना में अधिक जानकारी लाई है।" --- जॉन टुकी

डेटा वैज्ञानिक का एक हिस्सा यह दिखाना है कि वे जिस डेटा के साथ काम कर रहे हैं, उसकी गुणवत्ता और प्रकृति क्या है। ऐसा करने के लिए, वे अक्सर दिलचस्प विज़ुअलाइज़ेशन, या प्लॉट्स, ग्राफ़ और चार्ट बनाते हैं, जो डेटा के विभिन्न पहलुओं को दिखाते हैं। इस तरह, वे उन संबंधों और अंतरालों को दृश्य रूप से दिखा सकते हैं जिन्हें अन्यथा समझना मुश्किल होता है।

विज़ुअलाइज़ेशन यह तय करने में भी मदद कर सकते हैं कि डेटा के लिए कौन-सी मशीन लर्निंग तकनीक सबसे उपयुक्त है। उदाहरण के लिए, एक स्कैटरप्लॉट जो एक रेखा का अनुसरण करता हुआ प्रतीत होता है, यह संकेत देता है कि डेटा एक लीनियर रिग्रेशन अभ्यास के लिए उपयुक्त हो सकता है।

R कई ग्राफ बनाने की प्रणालियाँ प्रदान करता है, लेकिन [`ggplot2`](https://ggplot2.tidyverse.org/index.html) सबसे सुंदर और सबसे बहुमुखी में से एक है। `ggplot2` आपको **स्वतंत्र घटकों को संयोजित करके** ग्राफ बनाने की अनुमति देता है।

आइए Price और Month कॉलम के लिए एक साधारण स्कैटर प्लॉट से शुरू करें।

इस मामले में, हम [`ggplot()`](https://ggplot2.tidyverse.org/reference/ggplot.html) से शुरू करेंगे, एक डेटा सेट और एस्थेटिक मैपिंग (with [`aes()`](https://ggplot2.tidyverse.org/reference/aes.html)) प्रदान करेंगे, और फिर स्कैटर प्लॉट के लिए एक लेयर जोड़ेंगे (जैसे [`geom_point()`](https://ggplot2.tidyverse.org/reference/geom_point.html))।


In [ ]:
# Set a theme for the plots
theme_set(theme_light())

# Create a scatter plot
p <- ggplot(data = new_pumpkins, aes(x = Price, y = Month))
p + geom_point()

क्या यह प्लॉट उपयोगी है 🤷? क्या इसमें कुछ आपको चौंकाता है?

यह खास उपयोगी नहीं है क्योंकि यह केवल आपके डेटा को एक दिए गए महीने में बिंदुओं के फैलाव के रूप में दिखाता है।  


### **हम इसे उपयोगी कैसे बनाएं?**

चार्ट्स में उपयोगी डेटा दिखाने के लिए, आमतौर पर आपको डेटा को किसी न किसी तरीके से समूहित करना पड़ता है। उदाहरण के लिए, हमारे मामले में, हर महीने के लिए कद्दू की औसत कीमत ढूंढना हमारे डेटा में छिपे पैटर्न को बेहतर तरीके से समझने में मदद करेगा। यह हमें **dplyr** के एक और महत्वपूर्ण उपयोग की ओर ले जाता है:

#### `dplyr::group_by() %>% summarize()`

R में समूहित एग्रीगेशन को आसानी से निम्नलिखित तरीके से किया जा सकता है:

`dplyr::group_by() %>% summarize()`

-   `dplyr::group_by()` विश्लेषण की इकाई को पूरे डेटा सेट से बदलकर व्यक्तिगत समूहों, जैसे कि प्रति माह, में बदल देता है।

-   `dplyr::summarize()` एक नया डेटा फ्रेम बनाता है, जिसमें प्रत्येक समूहिंग वेरिएबल के लिए एक कॉलम और आपके द्वारा निर्दिष्ट किए गए प्रत्येक सारांश सांख्यिकी के लिए एक कॉलम होता है।

उदाहरण के लिए, हम `dplyr::group_by() %>% summarize()` का उपयोग करके कद्दुओं को **Month** कॉलम के आधार पर समूहों में विभाजित कर सकते हैं और फिर प्रत्येक महीने के लिए **औसत कीमत** निकाल सकते हैं।


In [ ]:
# Find the average price of pumpkins per month
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price))

संक्षेप में!✨

महीनों जैसे श्रेणीबद्ध फीचर्स को बार प्लॉट 📊 के जरिए बेहतर तरीके से प्रस्तुत किया जा सकता है। बार चार्ट के लिए जिम्मेदार लेयर्स हैं `geom_bar()` और `geom_col()`। अधिक जानकारी के लिए `?geom_bar` देखें।

चलो एक बनाते हैं!


In [ ]:
# Find the average price of pumpkins per month then plot a bar chart
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price)) %>% 
  ggplot(aes(x = Month, y = mean_price)) +
  geom_col(fill = "midnightblue", alpha = 0.7) +
  ylab("Pumpkin Price")

🤩🤩यह एक अधिक उपयोगी डेटा विज़ुअलाइज़ेशन है! ऐसा प्रतीत होता है कि कद्दू की सबसे अधिक कीमत सितंबर और अक्टूबर में होती है। क्या यह आपकी अपेक्षा के अनुरूप है? क्यों या क्यों नहीं?

दूसरा पाठ समाप्त करने पर बधाई 👏! आपने मॉडल निर्माण के लिए अपना डेटा तैयार किया, फिर विज़ुअलाइज़ेशन का उपयोग करके अधिक अंतर्दृष्टि प्राप्त की!



---

**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। जबकि हम सटीकता के लिए प्रयासरत हैं, कृपया ध्यान दें कि स्वचालित अनुवाद में त्रुटियां या अशुद्धियां हो सकती हैं। मूल भाषा में उपलब्ध मूल दस्तावेज़ को आधिकारिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सिफारिश की जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम उत्तरदायी नहीं हैं।  
